In [27]:
from pymongo import MongoClient
import os
from pprint import pprint
import json
from dotenv import load_dotenv

In [28]:
load_dotenv()

True

In [19]:
mongo_username = "root"
mongo_password = os.environ.get("PWPROD")
host = "localhost"
port = "27020"  # prod

mongoclient_prod = MongoClient(
    "mongodb://%s:%s@%s:%s" % (mongo_username, mongo_password, host, port)
)

In [30]:
mongo_username = "root"
mongo_password = os.environ.get("PWDEV")
host = "localhost"
port = "27019"  # dev

mongoclient_dev = MongoClient(
    "mongodb://%s:%s@%s:%s" % (mongo_username, mongo_password, host, port)
)

In [21]:
host = "localhost"
port = "27017"  # local

mongoclient_local = MongoClient("mongodb://%s:%s" % (host, port))

## For Tornado


In [22]:
def add_field(mongodbclient: MongoClient, collection: str) -> None:
    for document in mongodbclient["hazarddb"][collection].find():
        document["hazardDatasets"] = [
            {
                "datasetId": document["datasetId"],
                "demandType": "wind",
                "demandUnits": "mph",
                "threshold": None,
            }
        ]
        # replace document
        mongodbclient["hazarddb"][collection].replace_one(
            {"_id": document["_id"]}, document
        )
        # pprint(document)

In [34]:
def revert_add_field(mongodbclient: MongoClient, collection: str) -> None:
    for document in mongodbclient["hazarddb"][collection].find():
        del document["hazardDatasets"]
        # replace document
        mongodbclient["hazarddb"][collection].replace_one(
            {"_id": document["_id"]}, document
        )

In [31]:
## for local
# add_field(mongoclient_local, "TornadoDataset")

## for prod
# add_field(mongoclient_prod, "TornadoDataset")

## for dev
add_field(mongoclient_dev, "TornadoDataset")

In [36]:
## for local
# add_field(mongoclient_local, "TornadoModel")

## for prod
# add_field(mongoclient_prod, "TornadoModel")

## for dev
add_field(mongoclient_dev, "TornadoModel")
# revert_add_field(mongoclient_dev, "TornadoModel")

In [52]:
def del_datasetId(mongodbclient: MongoClient, collection: str, service: str) -> None:
    revert_dict: dict = {}
    for document in mongodbclient["hazarddb"][collection].find():
        # remove datasetId key from main document
        revert_dict[str(document["_id"])] = document["datasetId"]
        del document["datasetId"]
        # replace document
        mongodbclient["hazarddb"][collection].replace_one(
            {"_id": document["_id"]}, document
        )
    with open(f"revert_deleted_datasetIds_{collection}_{service}.json", "w") as f:
        f.write(json.dumps(revert_dict, indent=4))

In [43]:
def revert_del_datasetIds(
    mongodbclient: MongoClient, collection: str, service: str
) -> None:
    with open(f"revert_deleted_datasetIds_{collection}_{service}.json", "r") as f:
        revert_dict = json.load(f)
    for _id, datasetId in revert_dict.items():
        mongodbclient["hazarddb"][collection].update_one(
            {"_id": _id}, {"$set": {"datasetId": datasetId}}
        )

In [51]:
### run after we have updated all other components and we are ready to remove this key.

## for local
# del_datasetId(mongoclient_local, "TornadoDataset", "local")

## for prod
# del_datasetId(mongoclient_prod, "TornadoDataset", "prod")

## for dev
# del_datasetId(mongoclient_dev, "TornadoDataset", "dev")

###### TornadoModel

## for local
# del_datasetId(mongoclient_local, "TornadoModel", "local")

## for prod
# del_datasetId(mongoclient_prod, "TornadoModel", "prod")

## for dev
# del_datasetId(mongoclient_dev, "TornadoModel", "dev")

In [ ]:
### NOOO!!! revertttttt

## for local
# revert_del_datasetIds(mongoclient_local, "TornadoDataset", "local")

## for prod
# revert_del_datasetIds(mongoclient_prod, "TornadoDataset", "prod")

## for dev
# revert_del_datasetIds(mongoclient_dev, "TornadoDataset", "dev")

###### TornadoModel

## for local
# revert_del_datasetIds(mongoclient_local, "TornadoModel", "local")

## for prod
# revert_del_datasetIds(mongoclient_prod, "TornadoModel", "prod")

## for dev
# revert_del_datasetIds(mongoclient_dev, "TornadoModel", "dev")

## For Earthquake


In [37]:
def modify_field(mongodbclient: MongoClient) -> None:
    for document in mongodbclient["hazarddb"]["EarthquakeModel"].find():
        # modify field
        document["hazardDatasets"] = [document["rasterDataset"]]
        document.pop("rasterDataset")
        # replace document
        mongodbclient["hazarddb"]["EarthquakeModel"].replace_one(
            {"_id": document["_id"]}, document
        )

In [39]:
def revert_modify_field(mongodbclient: MongoClient) -> None:
    for document in mongodbclient["hazarddb"]["EarthquakeModel"].find():
        # modify field
        document["rasterDataset"] = document["hazardDatasets"][0]
        document.pop("hazardDatasets")
        # replace document
        mongodbclient["hazarddb"]["EarthquakeModel"].replace_one(
            {"_id": document["_id"]}, document
        )

In [41]:
## for local
# modify_field(mongoclient_local)

## for prod
# modify_field(mongoclient_prod)

## for dev
modify_field(mongoclient_dev)
# revert_modify_field(mongoclient_dev)